In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path_dir = '/content/drive/MyDrive/ComputerVision/HumanAndHorse'
link_data = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip'

In [ ]:
!pip install wget

In [ ]:
import wget
wget.download(link_data, path_dir)

'/content/drive/MyDrive/ComputerVision/HumanAndHorse/horse-or-human.zip'

In [3]:
%cd SwitchFrequencyAnalysis

[Errno 2] No such file or directory: 'SwitchFrequencyAnalysis'
/content


In [4]:
cd '/content/drive/MyDrive/ComputerVision/HumanAndHorse'

/content/drive/MyDrive/ComputerVision/HumanAndHorse


In [5]:
data = './HumanAndHorse'

In [ ]:

!unzip -q horse-or-human.zip -d $data

In [48]:
from keras.preprocessing.image import ImageDataGenerator

train_test = ImageDataGenerator(rescale=1/255, validation_split=0.1, horizontal_flip=True, width_shift_range=0.2, height_shift_range=0.2, rotation_range=0.15)

In [49]:
train_data = train_test.flow_from_directory(data, target_size=(150, 150), class_mode='binary', batch_size=90, subset='training')
valid_data = train_test.flow_from_directory(data, target_size=(150, 150), class_mode='binary', batch_size=90, subset='validation')

Found 925 images belonging to 2 classes.
Found 102 images belonging to 2 classes.


In [31]:
train_data.class_indices
valid_data.class_indices

{'horses': 0, 'humans': 1}

In [32]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

In [43]:
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 18496)            

In [11]:
import tensorflow as tf

In [46]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
batch_size = 90
model.fit_generator(train_data, steps_per_epoch=train_data.samples//batch_size, epochs=10, validation_data=valid_data, validation_steps=valid_data.samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
10/10 [==============================] - 13s 1s/step - loss: 0.1607 - accuracy: 0.9353 - val_loss: 0.2230 - val_accuracy: 0.8889
Epoch 2/10
10/10 [==============================] - 12s 1s/step - loss: 0.1527 - accuracy: 0.9473 - val_loss: 0.1834 - val_accuracy: 0.9111
Epoch 3/10
10/10 [==============================] - 12s 1s/step - loss: 0.1862 - accuracy: 0.9233 - val_loss: 0.2474 - val_accuracy: 0.8889
Epoch 4/10
10/10 [==============================] - 12s 1s/step - loss: 0.0786 - accuracy: 0.9767 - val_loss: 0.4506 - val_accuracy: 0.8000
Epoch 5/10
10/10 [==============================] - 12s 1s/step - loss: 0.2392 - accuracy: 0.9126 - val_loss: 0.2042 - val_accuracy: 0.9000
Epoch 6/10
10/10 [==============================] - 12s 1s/step - loss: 0.0899 - accuracy: 0.9665 - val_loss: 0.4036 - val_accuracy: 0.7889
Epoch 7/10
10/10 [==============================] - 12s 1s/step - loss: 0.1344 - accuracy: 0.9401 - val_loss: 0.2986 - val_accuracy: 0.8556
Epoch 8/10
10/10 [==

In [57]:
model.save('./model_horse_human.h5')

In [51]:
from google.colab import files
from keras.preprocessing import image

In [56]:
uploaded = files.upload()
for file in uploaded.keys():
  path = './' + file
  img = image.load_img(path, target_size=(150, 150))
  img = image.img_to_array(img).astype('float') * 1/ 255.0

  img = img.reshape(-1, 150, 150, 3)

  y_pred = model.predict(img)
  print(y_pred[0][0])
  if y_pred[0][0] > 0.5:
    print('Is Human')
  else:
    print('Is horse')

Saving human.jpg to human (1).jpg
1.0
Is Human
